In [1]:
import torch
from transformers import BarkModel, AutoProcessor
import soundfile as sf
import os
from IPython.display import Audio, display
def run_tts_pipeline(text_input, output_filename="output_audio.wav", model_name="suno/bark"):
    """
    기본 TTS 파이프라인을 실행합니다.

    Args:
        text_input (str): 음성으로 변환할 텍스트.
        output_filename (str): 생성된 오디오를 저장할 파일 이름.
        model_name (str): 사용할 Hugging Face TTS 모델의 이름.
    """
    print(f"\n--- TTS 파이프라인 시작: {model_name} ---")
    print(f"원본 텍스트: '{text_input}'")

    print(f"모델 로딩 중: {model_name}...")
    try:
        processor = AutoProcessor.from_pretrained(model_name)
        model = BarkModel.from_pretrained(model_name)
        print("모델 및 프로세서 로드 완료.")
    except Exception as e:
        print(f"모델 로드 중 오류 발생: {e}")
        print("설치된 모델 이름이 올바른지, 인터넷 연결이 되어 있는지 확인하세요.")
        print("또는 Hugging Face 모델 허브에 일시적인 문제가 있을 수 있습니다.")
        print(f"--- TTS 파이프라인 종료: {model_name} (오류 발생) ---\n")
        return

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    print(f"모델을 다음 장치에 로드했습니다: {device}")

    print(f"텍스트 인코딩 중: '{text_input}'")
    inputs = processor(text_input, return_tensors="pt").to(device)
    print("텍스트 인코딩 완료.")

    if 'input_ids' in inputs:
        print(f"인코딩된 input_ids.shape: {inputs['input_ids'].shape}")
        print(f"인코딩된 input_ids: {inputs['input_ids']}")
    else:
        print("경고: 인코딩된 입력에 'input_ids' 키가 없습니다. Bark 모델의 입력 형식을 확인하세요.")
        print(f"--- TTS 파이프라인 종료: {model_name} (입력 형식 오류) ---\n")
        return

    if inputs['input_ids'].numel() == 0 or inputs['input_ids'].shape[1] == 0:
        print("경고: 인코딩된 입력이 비어 있거나 시퀀스 길이가 0입니다. 유효한 텍스트를 입력했는지 확인하세요.")
        print(f"--- TTS 파이프라인 종료: {model_name} (빈 입력 또는 0 길이 시퀀스) ---\n")
        return

    print("음성 합성 시작...")
    try:
        with torch.no_grad():
            audio_output = model.generate(**inputs, do_sample=True)
        print("음성 합성 완료.")
    except Exception as e:
        print(f"음성 합성 중 오류 발생: {e}")
        print("모델 입력 차원 또는 내부 처리 문제일 수 있습니다.")
        print(f"--- TTS 파이프라인 종료: {model_name} (합성 오류) ---\n")
        return

    audio_data = audio_output.cpu().numpy().squeeze()
    sampling_rate = model.generation_config.sample_rate if hasattr(model.generation_config, 'sample_rate') else 24000

    if audio_data.size > 0:
        print(f"생성된 오디오 데이터 최소값: {audio_data.min():.4f}, 최대값: {audio_data.max():.4f}")
        if audio_data.max() - audio_data.min() < 1e-6:
            print("경고: 생성된 오디오 데이터의 값이 매우 작거나 모두 0에 가깝습니다. 소리가 들리지 않을 수 있습니다.")
    else:
        print("경고: 생성된 오디오 데이터가 비어 있습니다.")

    print(f"오디오를 다음 파일로 저장 중: {output_filename} (샘플링 레이트: {sampling_rate} Hz)")
    try:
        sf.write(output_filename, audio_data, sampling_rate)
        print(f"오디오 저장 완료: {os.path.abspath(output_filename)}")
    except Exception as e:
        print(f"오디오 저장 중 오류 발생: {e}")
        print(f"--- TTS 파이프라인 종료: {model_name} (저장 오류) ---\n")
        return

    print(f"--- TTS 파이프라인 성공적으로 종료: {model_name} ---\n")

if __name__ == "__main__":
    text_to_synthesize = "안녕하세요, 이것은 텍스트 음성 변환 테스트입니다."
    output_audio_file = "korean_tts_output.wav"
    korean_model_name = "suno/bark"

    print("\n========== 한국어 TTS 테스트 시작 ==========")
    run_tts_pipeline(text_to_synthesize, output_filename=output_audio_file, model_name=korean_model_name)
    print("========== 한국어 TTS 테스트 종료 ==========\n")

    if os.path.exists(output_audio_file):
        print(f"\n한국어 오디오 파일 '{output_audio_file}'이(가) 성공적으로 생성되었습니다.")
        display(Audio(output_audio_file, autoplay=False))
        print("위 오디오 플레이어를 통해 재생해 보세요.")
    else:
        print(f"\n경고: 한국어 오디오 파일 '{output_audio_file}'을(를) 찾을 수 없습니다. 파일 생성에 실패했을 수 있습니다.")


========== 한국어 TTS 테스트 시작 ==========

--- TTS 파이프라인 시작: suno/bark ---
원본 텍스트: '안녕하세요, 이것은 텍스트 음성 변환 테스트입니다.'
모델 로딩 중: suno/bark...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/353 [00:00<?, ?B/s]

speaker_embeddings_path.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

...

generation_config.json: 0.00B [00:00, ?B/s]

모델 및 프로세서 로드 완료.
모델을 다음 장치에 로드했습니다: cuda
텍스트 인코딩 중: '안녕하세요, 이것은 텍스트 음성 변환 테스트입니다.'
텍스트 인코딩 완료.
인코딩된 input_ids.shape: torch.Size([1, 256])
인코딩된 input_ids: tensor([[  9521, 118741,  35506,  24982,  48549,    117,  70347,   9867,  34994,
           9634,  17138,   9352,  51745,   9866,  34994,  58303,  48345,    119,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,    

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:10000 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


음성 합성 완료.
생성된 오디오 데이터 최소값: -0.9422, 최대값: 0.9681
오디오를 다음 파일로 저장 중: korean_tts_output.wav (샘플링 레이트: 24000 Hz)
오디오 저장 완료: /content/korean_tts_output.wav
--- TTS 파이프라인 성공적으로 종료: suno/bark ---

========== 한국어 TTS 테스트 종료 ==========


한국어 오디오 파일 'korean_tts_output.wav'이(가) 성공적으로 생성되었습니다.


위 오디오 플레이어를 통해 재생해 보세요.


TTS 실습

기본 TTS 실습: 텍스트 → 음성 생성

In [10]:
# 1) 의존성 설치
!pip install gTTS
!pip install playsound==1.2.2

# 2) TTS 변환 및 저장
from gtts import gTTS
from IPython.display import Audio

text = "안녕하세요. 이 시스템은 한국어 텍스트를 음성으로 변환합니다."
tts = gTTS(text=text, lang='ko')
tts.save("output.mp3")

Audio("output.mp3")

In [5]:
from gtts import gTTS
from IPython.display import Audio

word_list = ["사과", "바나나", "컴퓨터", "책상", "학교"]
text = " , ".join(word_list)
tts = gTTS(text="다음 단어를 따라 읽으세요: " + text, lang='ko')
tts.save("vocab.mp3")
Audio("vocab.mp3")

In [8]:
from gtts import gTTS
from IPython.display import Audio

# 직접 입력한 텍스트
text = "오늘의 뉴스 헤드라인: 한국 정부가 디지털 뉴딜 정책을 강화합니다."

# TTS 변환
tts = gTTS(text=text, lang='ko')
tts.save("news.mp3")

# 음성 재생
Audio("news.mp3")

In [4]:
menu = {
    1: "계좌 정보를 안내해 드리겠습니다.",
    2: "대출 상품에 대해 안내해 드리겠습니다.",
    3: "상담원과 연결하겠습니다."
}
choice = 2  # 사용자가 입력했다고 가정
response = menu.get(choice, "잘못된 입력입니다.")
tts = gTTS(text=response, lang='ko')
tts.save("ivr.mp3")
Audio("ivr.mp3")

실습

한국어로 녹음-> whisper로 SST구현 즉 녹음 파일을 text로 변환 -> 그 text를 gTTS로 음성파일 변환한다.

Audio("경로", autoplay=true)

In [1]:
# 필요한 패키지 설치
# !pip install transformers accelerate soundfile librosa
# 1) 의존성 설치
!pip install gTTS
!pip install playsound==1.2.2

# 2) TTS 변환 및 저장
from gtts import gTTS
from IPython.display import Audio

import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import librosa
import numpy as np
from base64 import b64decode
from google.colab.output import eval_js
from IPython.display import display
import time

# 1) 모델과 프로세서 로드
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2) 파일 단위 인식 함수
def transcribe_file(path): #음성 파일을 입력받아 그것을 출력하도록 하는 함수
    try:
        audio, sr = sf.read(path)
    except:
        audio, sr = librosa.load(path, sr=16000)  #wave form내 8kHz 안에 있는 data 보겠다.
    inputs = processor(audio, sampling_rate=sr, return_tensors="pt", padding=True)
    input_features = inputs.input_features.to(device)

    # Whisper가 기대하는 입력 길이 계산(최대 30s)
    encoder = model.model.encoder
    stride1 = encoder.conv1.stride[0]
    stride2 = encoder.conv2.stride[0]
    expected_len = model.config.max_source_positions * stride1 * stride2

    seq_len = input_features.shape[-1]
    if seq_len < expected_len:
        pad_len = expected_len - seq_len
        input_features = torch.nn.functional.pad(input_features, (0, pad_len))
    elif seq_len > expected_len:
        input_features = input_features[..., :expected_len]

    # attention_mask 생성
    attention_mask = torch.ones(input_features.shape[:-1], device=device)

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features=input_features,
            attention_mask=attention_mask
        )
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# 3) 마이크 권한 요청
print("마이크 권한 요청 중…")
eval_js("""
  navigator.mediaDevices.getUserMedia({ audio: true })
    .then(stream => stream.getTracks().forEach(t => t.stop()))
    .catch(e => console.error(e));
""")

# 4) 녹음 및 인식 함수 (UI 개선)
def record_and_transcribe_ui(filename="recorded_audio.webm", duration=3):
    status = display("준비 중", display_id=True)
    status.update("🔴 녹음 중")
    js = f"""
    async function recordAudio() {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const recorder = new MediaRecorder(stream);
      const chunks = [];
      recorder.ondataavailable = e => chunks.push(e.data);
      recorder.start();
      await new Promise(r => setTimeout(r, {duration * 1000}));
      recorder.stop();
      await new Promise(r => recorder.onstop = r);
      const blob = new Blob(chunks);
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      return await new Promise(res => reader.onloadend = () => res(reader.result.split(',')[1]));
    }}
    recordAudio();
    """
    try:
        b64data = eval_js(js)
        audio_bytes = b64decode(b64data)
        with open(filename, "wb") as f:
            f.write(audio_bytes)

        # 인식 진행 표시
        for i in range(6):
            status.update("⌛ 인식 중" + "." * ((i % 3) + 1))
            time.sleep(0.5)

        result = transcribe_file(filename)
        status.update(f"✅ 인식 결과: {result}")
        return result

    except Exception as e:
        status.update(f"⚠️ 오류 발생: {e}")
        raise


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 7.7 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

마이크 권한 요청 중…


In [14]:
print("이것은 실습 예제문입니다.")
result_Text=record_and_transcribe_ui(duration=3)
tts = gTTS(text=result_Text, lang='ko')
tts.save("TTS_result.mp3")
Audio("TTS_result.mp3",autoplay="true")

이것은 실습 예제문입니다.


'✅ 인식 결과:  이것은 실수 예분입니다.'

- **간단한 음성 입력 및 출력 문장 쌍 예시**
    - **입력:** "안녕하세요"
        - **출력:** "안녕하세요. 무엇을 도와드릴까요?"
    - **입력:** "커피 주문할게요"
        - **출력:** "네, 어떤 커피로 드릴까요?"
    - **입력:** "아메리카노 한 잔"
        - **출력:** "아메리카노 한 잔 맞으실까요?"
    - **입력:** "네"
        - **출력:** "알겠습니다. 잠시만 기다려 주세요."
    - **입력:** "결제할게요"
        - **출력:** "결제를 도와드리겠습니다."
    - **입력:** "취소해 주세요"
        - **출력:** "주문을 취소하시겠어요?"
    - **입력:** "감사합니다"
        - **출력:** "천만에요. 안녕히 가세요."

whisper모델로 SST 후 각 문자열에 대한 출력을 gTTS로 음성형태로 변환후 출력한다.

In [13]:
my_list=["안녕하세요. 무엇을 도와드릴까요?",
         "네, 어떤 커피로 드릴까요?",
         "아메리카노 한 잔 맞으실까요?",
         "알겠습니다. 잠시만 기다려 주세요.",
         "결제를 도와드리겠습니다.",
         "주문을 취소하시겠어요?",
         "천만에요. 안녕히 가세요."]
result_Text=record_and_transcribe_ui(duration=3)  #음성에 대한 text입력받음
if "안" in result_Text:
  tts = gTTS(text=my_list[0], lang='ko')
  print("출력:",my_list[0])
  tts.save("TTS_result.mp3")

elif "커" in result_Text:
  tts = gTTS(text=my_list[1], lang='ko')
  print("출력:",my_list[1])
  tts.save("TTS_result.mp3")

elif "아" in result_Text:
  tts = gTTS(text=my_list[2], lang='ko')
  print("출력:",my_list[2])
  tts.save("TTS_result.mp3")

elif result_Text=="네":
  tts = gTTS(text=my_list[3], lang='ko')
  print("출력:",my_list[3])
  tts.save("TTS_result.mp3")

elif "결" in result_Text:
  tts = gTTS(text=my_list[4], lang='ko')
  print("출력:",my_list[4])
  tts.save("TTS_result.mp3")

elif "취" in result_Text:
  tts = gTTS(text=my_list[5], lang='ko')
  print("출력:",my_list[5])
  tts.save("TTS_result.mp3")

elif "감" in result_Text:
  tts = gTTS(text=my_list[6], lang='ko')
  print("출력:",my_list[6])
  tts.save("TTS_result.mp3")


Audio("TTS_result.mp3",autoplay="true")

'✅ 인식 결과:  커피 주문할게요'

출력: 네, 어떤 커피로 드릴까요?


- **간단한 음성 명령 레시피 안내 시스템 실습: "라면 끓이기"**

**[목표]**

사용자가 음성 명령(텍스트 입력으로 시뮬레이션)을 하면, 시스템이 레시피 단계를 음성으로 안내해 줍니다. "다음" 명령으로 단계를 넘어가고, "반복" 명령으로 현재 단계를 다시 듣습니다.

In [1]:
# 1) 의존성 설치
!pip install gTTS
!pip install playsound==1.2.2

# 2) TTS 변환 및 저장
from gtts import gTTS
from IPython.display import Audio

import warnings
warnings.filterwarnings("ignore")

import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import soundfile as sf
import librosa
import numpy as np
from base64 import b64decode
from google.colab.output import eval_js
from IPython.display import display
import time

# 1) 모델과 프로세서 로드
processor = WhisperProcessor.from_pretrained("openai/whisper-small")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 2) 파일 단위 인식 함수
def transcribe_file(path): #음성 파일을 입력받아 그것을 출력하도록 하는 함수
    try:
        audio, sr = sf.read(path)
    except:
        audio, sr = librosa.load(path, sr=16000)  #wave form내 8kHz 안에 있는 data 보겠다.
    inputs = processor(audio, sampling_rate=sr, return_tensors="pt", padding=True)
    input_features = inputs.input_features.to(device)

    # Whisper가 기대하는 입력 길이 계산(최대 30s)
    encoder = model.model.encoder
    stride1 = encoder.conv1.stride[0]
    stride2 = encoder.conv2.stride[0]
    expected_len = model.config.max_source_positions * stride1 * stride2

    seq_len = input_features.shape[-1]
    if seq_len < expected_len:
        pad_len = expected_len - seq_len
        input_features = torch.nn.functional.pad(input_features, (0, pad_len))
    elif seq_len > expected_len:
        input_features = input_features[..., :expected_len]

    # attention_mask 생성
    attention_mask = torch.ones(input_features.shape[:-1], device=device)

    with torch.no_grad():
        predicted_ids = model.generate(
            input_features=input_features,
            attention_mask=attention_mask
        )
    return processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# 3) 마이크 권한 요청
print("마이크 권한 요청 중…")
eval_js("""
  navigator.mediaDevices.getUserMedia({ audio: true })
    .then(stream => stream.getTracks().forEach(t => t.stop()))
    .catch(e => console.error(e));
""")

# 4) 녹음 및 인식 함수 (UI 개선)
def record_and_transcribe_ui(filename="recorded_audio.webm", duration=3):
    status = display("준비 중", display_id=True)
    status.update("🔴 녹음 중")
    js = f"""
    async function recordAudio() {{
      const stream = await navigator.mediaDevices.getUserMedia({{ audio: true }});
      const recorder = new MediaRecorder(stream);
      const chunks = [];
      recorder.ondataavailable = e => chunks.push(e.data);
      recorder.start();
      await new Promise(r => setTimeout(r, {duration * 1000}));
      recorder.stop();
      await new Promise(r => recorder.onstop = r);
      const blob = new Blob(chunks);
      const reader = new FileReader();
      reader.readAsDataURL(blob);
      return await new Promise(res => reader.onloadend = () => res(reader.result.split(',')[1]));
    }}
    recordAudio();
    """
    try:
        b64data = eval_js(js)
        audio_bytes = b64decode(b64data)
        with open(filename, "wb") as f:
            f.write(audio_bytes)

        # 인식 진행 표시
        for i in range(6):
            status.update("⌛ 인식 중" + "." * ((i % 3) + 1))
            time.sleep(0.5)

        result = transcribe_file(filename)
        status.update(f"✅ 인식 결과: {result}")
        return result

    except Exception as e:
        status.update(f"⚠️ 오류 발생: {e}")
        raise


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.7 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


preprocessor_config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json: 0.00B [00:00, ?B/s]

마이크 권한 요청 중…


In [2]:
my_list=[
    "안녕하세요. 라면 레시피 안내를 시작할까요?",
    "네, 1단계입니다. 냄비에 물 550ml를 넣고 끓여주세요.",
    "네, 다음 단계입니다. 물이 끓으면 면과 수프, 건더기를 넣고 4분 30초간 끓입니다.",
    "네, 마지막 단계입니다. 계란이나 파를 추가하고 맛있게 드세요.",
    "첫 번째 단계입니다. 더 이상 이전 단계는 없습니다.",
    "마지막 단계입니다. 요리가 완성되었습니다.",
    "현재 2단계 조리 중입니다. 물이 끓으면 면과 수프, 건더기를 넣고 4분 30초간 끓입니다.",
    "레시피 안내를 종료합니다. 맛있게 드세요!",
    "죄송합니다. 제가 이해하지 못했습니다. '시작', '다음', '반복', '종료' 중 하나를 말씀해 주세요."
]
def sound_func(index):
  result_t=my_list[index]
  print(result_t)
  tts = gTTS(text=result_t, lang='ko')
  tts.save("TTS_result.mp3")

index=0
while index<7:#"레시피 안내를 종료합니다. 맛있게 드세요!" 이 문장이 my_list[7]에 있으므로 index가 7이상이 되면 종료시킴
  user_comm=record_and_transcribe_ui(duration=2) #SST
  if "시" in user_comm: #user_comm이라는 문자열에 '시'라는 단어가 있는 경우
    index=0
    sound_func(index)
  elif "다" in user_comm:
    index+=1  #list의 다음 문자열 가리키고 출력
    sound_func(index)
  elif "반" in user_comm: #list의 현재 문자열 출력
    sound_func(index)
  elif "종" in user_comm: #index를 7이상으로 높여 종료
    index=7
    sound_func(index)
  else: #이상한 문자열이 입력된 경우
    sound_func(8)

  display(Audio("TTS_result.mp3"))

'✅ 인식 결과:  시작하겠습니다.'

Using custom `forced_decoder_ids` from the (generation) config. This is deprecated in favor of the `task` and `language` flags/config options.
Transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English. This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`. See https://github.com/huggingface/transformers/pull/28687 for more details.


안녕하세요. 라면 레시피 안내를 시작할까요?


'✅ 인식 결과:  인사하고 계세요. 다음으로'

네, 1단계입니다. 냄비에 물 550ml를 넣고 끓여주세요.


...

네, 1단계입니다. 냄비에 물 550ml를 넣고 끓여주세요.


'✅ 인식 결과:  다음으로'

네, 다음 단계입니다. 물이 끓으면 면과 수프, 건더기를 넣고 4분 30초간 끓입니다.


'✅ 인식 결과:  종료해주세요'

레시피 안내를 종료합니다. 맛있게 드세요!
